In [1]:
!pip install transformers==4.42.4 datasets tensorflow==2.17.0 tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 745.3 kB/s eta 0:00:00 0:00:01
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.0 MB/s eta 0:00:0000:0100:14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
  Attempting uninstall: tensorflow
    Found existing installat

In [2]:
import os
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import TFAutoModel, AutoTokenizer

2024-09-05 14:40:31.910781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 14:40:31.937373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 14:40:31.944941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_name = "google/mobilebert-uncased"
model = TFAutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/164M [00:00<?, ?B/s]

I0000 00:00:1725547241.896440      36 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725547241.896856      36 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725547241.907932      36 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725547241.908327      36 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
dataset = load_dataset('dair-ai/emotion', 'split')

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True,max_length=55)

In [7]:
tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [10]:
BATCH_SIZE = 8

In [11]:
example = tokenized_dataset['train'][1]
data = list(example.values())

In [12]:
print("Input IDs:", data[1])

Input IDs: tf.Tensor(
[  101  1045  2064  2175  2013  3110  2061 20625  2000  2061  9636 17772
  2074  2013  2108  2105  2619  2040 14977  1998  2003  8300   102     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0], shape=(55,), dtype=int64)


In [13]:
print("Label:", data[0])

Label: tf.Tensor(0, shape=(), dtype=int64)


In [14]:
print("Attention Mask:", data[3])

Attention Mask: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(55,), dtype=int64)


In [15]:
print("token_type_ids :", data[2])

token_type_ids : tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(55,), dtype=int64)


In [16]:
def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[3],
        'token_type_ids': data[2]
    }, data[0]

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['train'][:])
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['validation'][:])
validation_dataset = validation_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [18]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(8, 55), dtype=int64, numpy=
array([[  101,  1045,  2514,  2066,  1045,  1049,  2008,  6530, 11669,
         8026,  2006,  1996,  4534,  2008,  6343,  2428,  4912,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [  101,  1045,  2514,  2066,  2008,  1055,  2061,  6881,  2008,
         1045,  2018,  4456,  2008,  2028,  2051,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [  101,  1045,  2514,  2004,  2295,  1045,  2572

In [19]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes, dropout_rate=0.3, l2_reg=0.01):
        super().__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(l2_reg))

    def call(self, inputs):
        x = self.bert(inputs)[1]
        x = self.dropout(x)
        return self.fc(x)

In [20]:
classifier = BERTForClassification(model, num_classes=6, dropout_rate=0.3, l2_reg=0.005)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [21]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [22]:
history = classifier.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=500,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/500


I0000 00:00:1725547482.883785     118 service.cc:146] XLA service 0x7d95dc056480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725547482.883824     118 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725547482.883830     118 service.cc:154]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1725547483.024642     118 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2000/2000 [==============================] - 856s 285ms/step - loss: 589491.5000 - accuracy: 0.1532 - val_loss: 29.2393 - val_accuracy: 0.3060
Epoch 2/500
2000/2000 [==============================] - 547s 273ms/step - loss: 498.5681 - accuracy: 0.2473 - val_loss: 3.2603 - val_accuracy: 0.3615
Epoch 3/500
2000/2000 [==============================] - 545s 272ms/step - loss: 16.8466 - accuracy: 0.2720 - val_loss: 2.4069 - val_accuracy: 0.3610
Epoch 4/500
2000/2000 [==============================] - 544s 272ms/step - loss: 5.6288 - accuracy: 0.2673 - val_loss: 2.1253 - val_accuracy: 0.3530
Epoch 5/500
2000/2000 [==============================] - 555s 277ms/step - loss: 4.2242 - accuracy: 0.2544 - val_loss: 2.0623 - val_accuracy: 0.3620
Epoch 6/500
2000/2000 [==============================] - 544s 272ms/step - loss: 3.5964 - accuracy: 0.2632 - val_loss: 1.9825 - val_accuracy: 0.3800
Epoch 7/500
2000/2000 [==============================] - 543s 272ms/step - loss: 3.3514 - accuracy: 0.2767 - 

In [23]:
evaluation_results = classifier.evaluate(test_dataset)

250/250 [==============================] - 28s 111ms/step - loss: 0.3566 - accuracy: 0.9195


In [24]:
base_save_path = './model'
os.makedirs(base_save_path, exist_ok=True)
# base_save_path = '/content/drive/MyDrive/Projects/ytproject/new_train_model'

In [25]:
model_save_path_tf = os.path.join(base_save_path, 'transformer')
classifier.save(model_save_path_tf, save_format='tf')

In [26]:
tokenizer_save_path = os.path.join(base_save_path, 'saved_tokenizer')
tokenizer.save_pretrained(tokenizer_save_path)

('./model/saved_tokenizer/tokenizer_config.json',
 './model/saved_tokenizer/special_tokens_map.json',
 './model/saved_tokenizer/vocab.txt',
 './model/saved_tokenizer/added_tokens.json',
 './model/saved_tokenizer/tokenizer.json')

In [27]:
results_save_path = os.path.join(base_save_path, 'evaluation_results.txt')
with open(results_save_path, 'w') as f:
    f.write(f"Loss: {evaluation_results[0]}\n")
    f.write(f"Accuracy: {evaluation_results[1]}\n")